# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [15]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')}) # what is  re.compile('.*') ?
for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 復仇母親, ID: 10392
Movie: 風暴過後, ID: 10387
Movie: 南方車站的聚會, ID: 10380
Movie: 祭旗, ID: 10373
Movie: 唯你是愛, ID: 10370
Movie: 屍控療法, ID: 10367
Movie: 黑熊來了, ID: 10357
Movie: 唐人街探案2, ID: 10353
Movie: 黑計畫, ID: 10351
Movie: HUMAN LOST人間失格, ID: 10348
Movie: 假面騎士劇場版ZI-O Over Quartzer, ID: 10331
Movie: 巴黎小情聖, ID: 10329
Movie: 每個月來訪一次的月經醬, ID: 10324
Movie: 8級警戒, ID: 10323
Movie: 為你存在的每一天, ID: 10297
Movie: 黑色聖誕節, ID: 10291
Movie: 熱氣球飛行家, ID: 10257
Movie: 大說謊家, ID: 10122
Movie: 特約經紀公司, ID: 10364
Movie: 一屍到底番外篇：好萊塢大作戰！, ID: 10352
Movie: 花生醬獵鷹的願望, ID: 10340
Movie: 女孩要革命, ID: 10339
Movie: 我的靈魂是愛做的, ID: 10336
Movie: 絕世情歌, ID: 10328
Movie: 人間失格：太宰治與他的3個女人, ID: 10287
Movie: 抓狂博物館, ID: 10279
Movie: 呆萌特務, ID: 10264
Movie: 去年聖誕節, ID: 10256
Movie: 野蠻遊戲：全面晉級, ID: 10134
Movie: 夕霧花園, ID: 10327
Movie: 寂寞診療室, ID: 10282
Movie: 懸案密碼前傳：瓶中信, ID: 10254
Movie: 布魯克林孤兒, ID: 10224
Movie: 暴走曼哈頓, ID: 10044
Movie: 鋒迴路轉, ID: 10258
Movie: 賽道狂人, ID: 10097
Movie: 麂皮：永不滿足, ID: 10358
Movie: 82年生的金智英, ID: 10298
Movie: 冰雪奇緣2, ID: 959

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [76]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10392

In [77]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater' # 有人問：yahoo movie api
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [78]:
# 指定放映地區
area_id = 28

In [79]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 13
十二月 14
十二月 15
十二月 16
十二月 17


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [80]:
# 選定要觀看的日期
date = "2019-12-13"

In [81]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [92]:
html[0] # 解析 範例

<ul class="area_time _c jq_area_time" data-theater_name="台北in89豪華數位影城" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=36" data-theater_url="http://www.in89.com.tw/website_module.php?website_module_classify_sn=39" id="theater_id_36">
<li class="adds">
<a href="https://movies.yahoo.com.tw/theater_result.html/id=36">台北in89豪華數位影城</a>
<span>02-23315077</span>
</li>
<li class="taps">
<span class="tapR">數位</span> </li>
<li class="time _c">
<div class="input_picker jq_input_picker">
<input class="gabtn" data-cinema="e9c365f27f4e11e494ae8b4ddf78b93a" data-date="1576166400" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '復仇母親']" data-movie="e6c496a3aaf5458d965d4639a6479293" data-movie_date="12.13" data-movie_time="11:30" data-movie_title="復仇母親" data-movie_type="數位" data-session="4ad3572edd5f440ca790dadced90c99a" data-time="1576207800" id="72577057" name="schedule_list" type="radio" value="2019-12-13 11:30:00"/>
<label class="select" for="72577057">11:30</labe

In [94]:
html[0]['data-theater_name'],html[0].a.text

('台北in89豪華數位影城', '台北in89豪華數位影城')

In [115]:
html[0].find(class_='tapR').text , type(html[0].find(class_='gabtn')) # tag 就要去指定key

('數位', bs4.element.Tag)

In [128]:
html[0].find(class_='gabtn') # find_all can use --> so list

<input class="gabtn" data-cinema="e9c365f27f4e11e494ae8b4ddf78b93a" data-date="1576166400" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '復仇母親']" data-movie="e6c496a3aaf5458d965d4639a6479293" data-movie_date="12.13" data-movie_time="11:30" data-movie_title="復仇母親" data-movie_type="數位" data-session="4ad3572edd5f440ca790dadced90c99a" data-time="1576207800" id="72577057" name="schedule_list" type="radio" value="2019-12-13 11:30:00"/>

In [111]:
html[0].find(class_='gabtn')['data-movie_title'],html[0].find(class_='gabtn')['data-movie_type'],html[0].find(class_='gabtn')['value']

('復仇母親', '數位', '2019-12-13 11:30:00')

In [127]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''
for t in html:
    print("----------------------------------------------------------------------")
    theater_name = t.a.text
    print("電影名稱：",theater_name)
    t_content=t.find_all(class_='gabtn')
    for movie in t_content:
        movie_name = movie['data-movie_title']
        movie_style = movie['data-movie_type']
        movie_time = movie['value']
        print(movie_name,movie_style,movie_time)
    

----------------------------------------------------------------------
電影名稱： 台北in89豪華數位影城
復仇母親 數位 2019-12-13 11:30:00
復仇母親 數位 2019-12-13 17:50:00
復仇母親 數位 2019-12-13 22:05:00
----------------------------------------------------------------------
電影名稱： 喜滿客絕色影城
復仇母親 數位 2019-12-13 12:25:00
復仇母親 數位 2019-12-13 16:25:00
復仇母親 數位 2019-12-13 20:40:00
----------------------------------------------------------------------
電影名稱： 台北信義威秀影城
復仇母親 數位 2019-12-13 16:00:00
復仇母親 數位 2019-12-13 19:25:00
復仇母親 數位 2019-12-13 23:15:00
----------------------------------------------------------------------
電影名稱： 哈拉影城
復仇母親 數位 2019-12-13 13:10:00
復仇母親 數位 2019-12-13 18:10:00
復仇母親 數位 2019-12-13 22:10:00
復仇母親 數位 2019-12-14 00:10:00
----------------------------------------------------------------------
電影名稱： 百老匯數位影城
復仇母親 數位 2019-12-13 09:50:00
復仇母親 數位 2019-12-13 13:55:00
復仇母親 數位 2019-12-13 18:00:00
復仇母親 數位 2019-12-13 22:05:00
復仇母親 數位 2019-12-14 00:00:00
--------------------------------------------------------------------